Having completed data wrangling and EDA , it is now time to preprocess the split the data in order to make it usable for data modeling.

In [148]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


Import the CSV file and convert it to a dataframe.

In [150]:
df = pd.read_csv('df.csv', index_col=False)

Rename columns to remove metrics that become irrelevant after standardization.

In [152]:
df.rename( columns = {'NA Sales (Millions)': 'NA Sales', 'Global Sales (Millions)': 'Global Sales'}, inplace = True)

In [153]:
df.head()

,GameID,Game,Release Date,Rating,Genre,Price (USD),Platform,Publisher,Developer,Metacritic Score,Player Score,NA Sales,Global Sales
0,12140,Max Payne,2011-01-06,M,Action,3.49,PS2,Take-Two Interactive,Rockstar Toronto,89,8.5,1.99,3.31
1,12140,Max Payne,2011-01-06,M,Action,3.49,XB,Take-Two Interactive,neo Software,89,8.3,0.98,1.22
2,12140,Max Payne,2011-01-06,M,Action,3.49,GBA,Take-Two Interactive,Mobius Entertainment,89,7.8,0.02,0.03
3,748490,The Legend of Heroes: Trails of Cold Steel II,2018-02-14,T,RPG,39.99,PSV,Nippon Ichi Software,Falcom,81,7.3,0.02,0.19
4,748490,The Legend of Heroes: Trails of Cold Steel II,2018-02-14,T,RPG,39.99,PS3,Nippon Ichi Software,Falcom,81,7.8,0.02,0.15


Check on the datatypes of each column.

In [155]:
df.dtypes

GameID                int64
Game                 object
Release Date         object
Rating               object
Genre                object
Price (USD)         float64
Platform             object
Publisher            object
Developer            object
Metacritic Score      int64
Player Score        float64
NA Sales            float64
Global Sales        float64
dtype: object

Create a variable, dfo, with only the non-numerical columns.

In [157]:
dfo = df.select_dtypes(include = 'object')

Create dummy variables using dfo and reduce the amount of dummy variables created to increase model efficiency by putting in a threshold of .01.

In [159]:
df_dummies = pd.get_dummies(dfo)
selector = VarianceThreshold(threshold = .01)
df_dummies_reduced = selector.fit_transform(df_dummies)

Add the dummy variables to the original dataframe.

In [160]:
selected_columns = df_dummies.columns[selector.get_support()]
df_dummies_reduced = pd.DataFrame(df_dummies_reduced, columns=selected_columns)
df_reduced = df.drop(columns=dfo.columns)
df = pd.concat([df_reduced, df_dummies_reduced], axis=1)

Check on the new shape of the df with the dummy variables

In [162]:
df.shape

(442, 88)

Check the info of the new dataframe, such as the dtype's of the dummy variables.

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 88 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   GameID                                            442 non-null    int64  
 1   Price (USD)                                       442 non-null    float64
 2   Metacritic Score                                  442 non-null    int64  
 3   Player Score                                      442 non-null    float64
 4   NA Sales                                          442 non-null    float64
 5   Global Sales                                      442 non-null    float64
 6   Game_F.E.A.R.                                     442 non-null    bool   
 7   Game_Portal 2                                     442 non-null    bool   
 8   Game_The Evil Within                              442 non-null    bool   
 9   Game_Tomb Raider: Leg

Standardize the numeric variables for ML modeling.

In [211]:
# Select numeric columns, excluding GameID and Global Sales since GameID is an identifier and Global Sales is the depedent variable
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns.drop(['GameID'])

# Select boolean columns to keep them out of the standardizing process
boolean_columns = df.select_dtypes(include=['bool'])

# Keep the GameID and Global Sales columns separate
gid_gs_columns = df[['GameID']] 

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the numeric data and transform it
df_numeric_standardized = scaler.fit_transform(df[numeric_columns])

# Convert the resulting NumPy array back to a df with the same column names
df_numeric_standardized = pd.DataFrame(df_numeric_standardized, columns=numeric_columns)

#Merge the two dataframe's back together to create the complete dataframe
df_final = pd.concat([gid_gs_columns, df_numeric_standardized, boolean_columns], axis=1)

#Check the new dataframe
df_final.head()

,GameID,Price (USD),Metacritic Score,Player Score,NA Sales,Global Sales,Game_F.E.A.R.,Game_Portal 2,Game_The Evil Within,Game_Tomb Raider: Legend,...,Developer_Haemimont,Developer_Io Interactive,Developer_MachineGames,Developer_Obsidian Entertainment,Developer_Rebellion,Developer_Tango Gameworks,Developer_Techland,Developer_Valve Software,Developer_Volition Inc.,Developer_Yuke's
0,12140,-0.909379,1.175885,0.984675,3.028921,2.878773,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,12140,-0.909379,1.175885,0.834958,1.165682,0.591315,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,12140,-0.909379,1.175885,0.460666,-0.605317,-0.711113,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,748490,2.429418,0.491903,0.086375,-0.605317,-0.535997,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,748490,2.429418,0.491903,0.460666,-0.605317,-0.579776,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Seperate the indepdent variables from the depedent variable.

In [168]:
X = df_final.drop(columns = ['Global Sales', 'GameID'])
y = df_final['Global Sales']

Split the data into training and testing sections with a split of 80/20, both for the X and y.

In [170]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state= 42)

Check the shapes of the training and testing data to make sure it was split accurately.

In [172]:
X_train.shape


(353, 86)

In [173]:
X_test.shape

(89, 86)

In [174]:
y_train.shape

(353,)

In [175]:
y_test.shape

(89,)

Check the final dataframe now that its numeric columns have been standardized and its non-numeric columns have been divided into dummy variables.

In [176]:
df_final.head()

,GameID,Price (USD),Metacritic Score,Player Score,NA Sales,Global Sales,Game_F.E.A.R.,Game_Portal 2,Game_The Evil Within,Game_Tomb Raider: Legend,...,Developer_Haemimont,Developer_Io Interactive,Developer_MachineGames,Developer_Obsidian Entertainment,Developer_Rebellion,Developer_Tango Gameworks,Developer_Techland,Developer_Valve Software,Developer_Volition Inc.,Developer_Yuke's
0,12140,-0.909379,1.175885,0.984675,3.028921,2.878773,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,12140,-0.909379,1.175885,0.834958,1.165682,0.591315,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,12140,-0.909379,1.175885,0.460666,-0.605317,-0.711113,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,748490,2.429418,0.491903,0.086375,-0.605317,-0.535997,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,748490,2.429418,0.491903,0.460666,-0.605317,-0.579776,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Save the new dataframe.

In [177]:
df_final.to_csv('df_final', index=False)

Now that the data has been standardized, split, and has incorporated dummy variables, it is now ready to be used for data modeling.